## Facebook Friend

In [ ]:
'''
@ Title: Exercise to give a list of common friends.
@ Author: Puxin Xu
@ Time: at 00:40 am EST, Sep 15 Wed, 2016
@ IDE: MobaXterm_v9.1
'''

from __future__ import print_function
import sys
import re
from random import random
from operator import add
from pyspark import SparkContext
from itertools import chain
if __name__ == "__main__":
    sc = SparkContext(appName="Facebook")
    data = sc.textFile("facebook_small.txt")
# Step1: to flatMap each line to pairs: (key, value), 
# key is trio one of which is the object in this line; value = 1
    def byTrios(line):
        List = []
        words = re.findall(r"[\w']+",line)
        for word in range(1,len(words)-1):
            for word2 in range(word+1,len(words)):
                friend = [words[0],words[word],words[word2]]
                friend.sort()
                List.append([(friend[0],friend[1],friend[2]),1])
        return List
    
    trios = data.flatMap(byTrios)
    #print(tripairs.take(20))

* This step is to do an operation as followed

![pic1](1.png)

* The reason to set the object and keep it in each trio key is to avoid the trio key like followed which will impact the next step.

![pic1](3.png)


In [ ]:
# Step2: to filter trios which occur less than 3 times    
    tris = trios.reduceByKey(add).filter(lambda x: x[1] == 3)
    #print(tris.take(40))

* This filter operation is based on the simple principle:

![pic1](2.png)

In [ ]:
# Step3: to transform one trio to three pairs as key and another one as value  
    def pairToOneMethod(tri):
        return [[(tri[0][0],tri[0][1]),tri[0][2]],[(tri[0][0],tri[0][2]),tri[0][1]],[(tri[0][1],tri[0][2]),tri[0][0]]]
    pairToOne = tris.flatMap(pairToOneMethod)
    #print(pairToOne.take(20)) 

* transform ($A, B, C$) to $(A, B) -> C$,  $(A, C) -> B$  and  $(B, C) -> A$
* because A, B, C are friends with each other.

In [ ]:
# Step4: to combine all friends for each key
    def groupMethod(a,b):
        if (type(a) is list) and (type(b) is list):
            out = list(chain(*[a,b]))
        elif (type(a) is list) and (type(b) is not list):
            out = list(chain(*[a,[b]]))
        elif (type(a) is not list) and (type(b) is list):
            out =list(chain(*[[a],b]))
        else:
            out = list(chain(*[[a],[b]]))
        out.sort()
        return out
           
    result = pairToOne.reduceByKey(groupMethod)
    SortedResult = result.sortBy(lambda x : x[0][0])    
    print(SortedResult.take(20))    
    sc.stop()  

* put all keys' (e.g. $(A, B)$) common friends together into values.